<a href="https://colab.research.google.com/github/cherrowo/Disney-Algo/blob/main/GENETIC_ALGO_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import numpy as np

data = pd.read_csv("Values for Disney Map - Nodes.csv")
adj_matrix = pd.read_csv("Values for Disney Map - FINAL AM(travel + wait for food + ride).csv", header=None).values

nodes = data['Node ID'].values
types = data['Type'].values
ratings = data['Attraction Rating (?/5)'].values
wait_times = data['Waiting Time'].values
durations = data['Duration of Experience'].values
normalized_wait_times = data['Normalized_wait time'].values


In [ ]:

def calculate_time(solution):
  total_time = 0
  for i in range(len(solution) - 1):
        current_node = solution[i]
        next_node = solution[i+1]

        current_index = np.where(nodes == current_node)[0][0]
        next_index = np.where(nodes == next_node)[0][0]

        travel_time = adj_matrix[current_index][next_index]

        total_time += travel_time + durations[current_index] + normalized_wait_times[current_index]
  return total_time

def calculate_fitness(solution):
    """Calculates the number of rides visited for a given solution."""
    # Ensure the solution starts and ends at node 0
    if solution[0] != nodes[0] or solution[-1] != nodes[0]:
        return 0  # Penalize invalid solutions

    total_time = calculate_time(solution)
    if total_time > 780:
        return 0 # Penalize itineraries that exceed the time limit

    # Count the number of unique rides visited (excluding the entrance/exit)
    return len(set(solution[1:-1]))

def generate_initial_population(population_size, num_rides):
    population = []
    for _ in range(population_size):
        solution = [nodes[0]]  # Start at node 0
        remaining_rides = list(nodes[1:])
        random.shuffle(remaining_rides)
        solution.extend(remaining_rides[:num_rides - 1])
        solution.append(nodes[0]) # End at node 0
        population.append(solution)
    return population


def mutate(solution):
    idx1, idx2 = random.sample(range(1, len(solution) - 1), 2)  # Exclude the entrance/exit nodes
    solution[idx1], solution[idx2] = solution[idx2], solution[idx1]
    return solution


def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child = parent1[:crossover_point] + [item for item in parent2 if item not in parent1[:crossover_point] and item != nodes[0]][:len(parent1) - len(parent1[:crossover_point])]
    child.append(nodes[0]) #Ensure end at node 0
    return child


def genetic_algorithm(population_size=50, generations=100, mutation_rate=0.1, num_rides=10):
    population = generate_initial_population(population_size, num_rides)
    best_solution = None
    best_fitness = 0

    for generation in range(generations):
        fitness_scores = [calculate_fitness(solution) for solution in population]

        best_solution_idx = np.argmax(fitness_scores) #Find max fitness
        if fitness_scores[best_solution_idx] > best_fitness:
            best_fitness = fitness_scores[best_solution_idx]
            best_solution = population[best_solution_idx]

        new_population = []
        for _ in range(population_size):
            parent1 = random.choices(population, weights=fitness_scores, k=1)[0]
            parent2 = random.choices(population, weights=fitness_scores, k=1)[0]
            child = crossover(parent1, parent2)

            if random.random() < mutation_rate:
                child = mutate(child)
            new_population.append(child)
        population = new_population
    print("Optimal solution:", best_solution)
    print("Number of rides visited:", best_fitness)
    print("Total time:", calculate_time(best_solution))
    return best_solution, best_fitness

In [ ]:
best_solution, best_fitness = genetic_algorithm()

Optimal solution: [0, 10, 5, 18, 58, 33, 62, 6, 1, 67, 38, 25, 45, 27, 9, 3, 46, 63, 19, 61, 0]
Number of rides visited: 19
Total time: 741.05374592828


In [ ]:
calculate_time(best_solution)

741.05374592828

In [ ]:
attraction_info = []

for node_id in best_solution:
    # Find the index of the node in the original dataframe
    node_index = data[data['Node ID'] == node_id].index[0]

    # Extract attraction name and type
    attraction_name = data.loc[node_index, 'Title']
    attraction_type = data.loc[node_index, 'Type']

    attraction_info.append((attraction_name, attraction_type))

print("\nAttraction Name and Type for the Best Solution:")
for name, type_ in attraction_info:
    print(f"{name}, Type: {type_}")


Attraction Name and Type for the Best Solution:
Walt Disney World Railroad 1, Type: attraction
Jungle Cruise, Type: attraction
Plaza Ice Cream Parlor, Type: food
Walt Disney World Railroad 2, Type: attraction
TRON Lightcycle/Run, Type: attraction
Prince Charming Regal Carousel, Type: attraction
Walt Disney's Carousel of Progress, Type: attraction
Main Street Bakery, Type: food
Town Square Theatre, Type: attraction
Cool Ship, Type: food
Under the Sea Journey of the Little Mermaid, Type: attraction
The Hall of Presidents, Type: attraction
Meet Merida at Fairytale Garden, Type: attraction
Haunted Mansion, Type: attraction
The Magic Carpets of Aladdin, Type: attraction
The Crystal Palace, Type: food
The Many Adventures of Winnie the Pooh, Type: attraction
Buzz Lightyear's Space Ranger Spin, Type: attraction
Big Thunder Mountain Railroad, Type: attraction
Tomorrowland Transit Authority Peoplemover, Type: attraction
Walt Disney World Railroad 1, Type: attraction
